In [1]:
import gdal
import numpy as np
from scipy import ndimage

# ETOPO1 GeoTIFF


GeoTIFF is the most versatile raster format for geo-referenced imagery and is the default for all `gdal` utilities.

We load the ETOPO1 GeoTiff that can be downloaded from NOAA website: 
**https://www.ngdc.noaa.gov/mgg/global/global.html**

In [15]:
dir_lvl = 'lvl_8'

In [16]:
gtiff = gdal.Open("data/ETOPO1.tif")

width = gtiff.RasterXSize
height = gtiff.RasterYSize
gt = gtiff.GetGeoTransform()
img = gtiff.GetRasterBand(1).ReadAsArray().T
img = np.fliplr(img)

# Mapping dataset on the sphere

Now we map the ETOPO1 elevation to the spherical mesh by interpolation using **scipy** [**`map_coordinates`**](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.ndimage.interpolation.map_coordinates.html.)

Our spherical mesh is defined on the unit sphere and the longitudes and latitudes range from [$-\pi,\pi$] and [-$\pi/2,\pi/2$] respectively. 

We first convert the points in degrees and map their position with respect to ETOPO1 dataset shape.

In [17]:
mesh_lonlat = np.load(dir_lvl+'/icoord_etopo.npz')

print("Field names: {}".format(mesh_lonlat.files))

icoords = mesh_lonlat['v']

Field names: ['v']


We then performe the interpolation...

In [18]:
elevations = ndimage.map_coordinates(img, icoords , order=3, mode='nearest').astype(np.float)

We now load the mesh information from the `A-create-base-mesh` notebook.

In [19]:
mesh_struct = np.load(dir_lvl+'/mesh.npz')

print("Field names: {}".format(mesh_struct.files))

cells = mesh_struct['c']
ngbIDs = mesh_struct['n']
vertices = mesh_struct['v']

Field names: ['c', 'n', 'v']


And now we can store the elevation mesh to run with the simulation...

In [20]:
etopofile = dir_lvl+'/etopo'
np.savez_compressed(etopofile, v=vertices, c=cells, 
                    n=ngbIDs.astype(int), z=elevations)

We can export the mesh as a `VTK` file to check everything work well!

In [8]:
import meshio

etopovtk = dir_lvl+"/etopo.vtk"
vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, point_data={"z":elevations})
meshio.write(etopovtk, vis_mesh)

print("Writing VTK file {}".format(etopovtk))

Writing VTK file lvl_9/etopo.vtk
